In [1]:
import os
import time
import boto3
import json


# Initialize AWS clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=region)


account_id_suffix = account_id[:3]
region_suffix = ''.join([word[0] for word in region.split('-')])
suffix = f"{region_suffix}-{account_id_suffix}"

# Define Knowledge Base parameters
knowledge_base_name_standard = 'fixed-size-chunk-kb'
knowledge_base_name_hierarchical = 'hierarchical-chunk-kb'
knowledge_base_description = "Knowledge Base containing sample synthetic Octank financial data"

print(f"AWS Region: {region}", f"Account ID: {account_id}", f"Suffix: {suffix}", sep='\n')

AWS Region: us-east-1
Account ID: 533267284022
Suffix: ue1-533


In [2]:
from utils.knowledgebase import BedrockKnowledgeBases
from utils.agents import BedrockAgents
from utils.agents import AWSResourceManager
from utils.chat import BedrockChat, SyntheticDataGenerator
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

    
kb = BedrockKnowledgeBases()
agents = BedrockAgents()

In [3]:
agent_name=f'rag-agent-{suffix}'
kb_id = "892DWWRLXW"

print(f"Agent name: {agent_name}", f"KB id: {kb_id}", sep='\n')


Agent name: rag-agent-ue1-533
KB id: 892DWWRLXW


In [4]:
user_query = "What is the total cost of Octank Financial's property and equipment as of December 31, 2022?"

result = agents.invoke(
    agent_name=agent_name, 
    input_text=user_query, 
    verbose=True, 
    trace_level='core',
    save_trace_json_file='agent_full_trace.json'
)


2025-01-07 16:52:18,091 [INFO] Invoking agent 'rag-agent-ue1-533' with input: What is the total cost of Octank Financial's property and equipment as of December 31, 2022?
---- Step 1 ----
Took 3.3s, using 3251 tokens (in: 3145, out: 106) to complete action.
Final response:
According to the table for Property and Equipment, Net under NOTES TO CONSOLIDATED FINANCIAL STATEMENTS, the total cost of Octank Financial's property and equipment as of December 31, 2022 is $440,000,000....
Agent made a total of 1 LLM calls, using 3251 tokens (in: 3145, out: 106), and took 3.8 total seconds.


In [5]:
print_markdown(result['response'])

According to the table for Property and Equipment, Net under NOTES TO CONSOLIDATED FINANCIAL STATEMENTS, the total cost of Octank Financial's property and equipment as of December 31, 2022 is $440,000,000.

In [6]:
simple_result = agents.simple_invoke(
    agent_name=agent_name, 
    input_text=user_query,
    verbose=True
)

2025-01-07 16:52:22,066 [INFO] Invoking agent 'rag-agent-ue1-533' with input: What is the total cost of Octank Financial's property and equipment as of December 31, 2022?
2025-01-07 16:52:22,179 [INFO] Agent invocation response: {'ResponseMetadata': {'RequestId': '507de721-fde1-48de-8f17-dc78e1b7bad5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 08 Jan 2025 00:52:22 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '507de721-fde1-48de-8f17-dc78e1b7bad5', 'x-amz-bedrock-agent-session-id': 'eb756e7b-ff1a-4b6e-9fe1-c20c81cc5f47', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': 'eb756e7b-ff1a-4b6e-9fe1-c20c81cc5f47', 'completion': <botocore.eventstream.EventStream object at 0x11772e070>}
2025-01-07 16:52:25,445 [INFO] Agent 'rag-agent-ue1-533' response: According to the table for Property and Equipment, Net under N

In [7]:
simple_result['trace']

[{'agentAliasId': 'TSTALIASID',
  'agentId': 'XNSPVCAHBV',
  'agentVersion': 'DRAFT',
  'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:533267284022:agent-alias/XNSPVCAHBV/TSTALIASID'}],
  'sessionId': 'eb756e7b-ff1a-4b6e-9fe1-c20c81cc5f47',
  'trace': {'orchestrationTrace': {'invocationInput': {'invocationType': 'KNOWLEDGE_BASE',
     'knowledgeBaseLookupInput': {'knowledgeBaseId': '892DWWRLXW',
      'text': "What is the total cost of Octank Financial's property and equipment as of December 31, 2022?"},
     'traceId': '507de721-fde1-48de-8f17-dc78e1b7bad5-0'}}}},
 {'agentAliasId': 'TSTALIASID',
  'agentId': 'XNSPVCAHBV',
  'agentVersion': 'DRAFT',
  'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:533267284022:agent-alias/XNSPVCAHBV/TSTALIASID'}],
  'sessionId': 'eb756e7b-ff1a-4b6e-9fe1-c20c81cc5f47',
  'trace': {'orchestrationTrace': {'observation': {'knowledgeBaseLookupOutput': {'retrievedReferences': [{'content': {'text': "The portfolio is classified into

In [5]:
user_query = "What is the total cost of Octank Financial's property and equipment as of December 31, 2022?"

result = agents.new_invoke(
    agent_name=agent_name, 
    input_text=user_query, 
    verbose=True, 
    trace_level='core',
)


2025-01-08 14:07:14,780 [INFO] Invoking agent 'rag-agent-ue1-533' with input: What is the total cost of Octank Financial's property and equipment as of December 31, 2022?
---- Step 1 ----
Took 3.5s, using 1825 tokens (in: 1679, out: 146) to complete action.
To find the total cost of Octank Financial's property and equipment as of December 31, 2022, I will need to search for information on Octank Financial's financial statements or reports. Let me query the knowledge base for relevant information.
---- Step 2 ----
Took 5.7s, using 2122 tokens (in: 2019, out: 103) to complete action.
Final response:


According to the table for Property and Equipment, Net under NOTES TO CONSOLIDATED FINANCIAL STATEMENTS, the total cost of Octank Financial's property and equipment as of December 31, 2022 was $440,000,000.


...
Agent made a total of 2 LLM calls, using 3947 tokens (in: 3698, out: 249), and took 9.9 total seconds.


In [5]:
result.keys()

dict_keys(['response', 'raw_trace', 'trace'])

In [6]:
result['trace']

In [7]:
result['raw_trace']

[{'agentAliasId': 'TSTALIASID',
  'agentId': 'XNSPVCAHBV',
  'agentVersion': 'DRAFT',
  'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:533267284022:agent-alias/XNSPVCAHBV/TSTALIASID'}],
  'sessionId': '562ea3f7-0945-456a-8760-4b534e460061',
  'trace': {'orchestrationTrace': {'invocationInput': {'invocationType': 'KNOWLEDGE_BASE',
     'knowledgeBaseLookupInput': {'knowledgeBaseId': '892DWWRLXW',
      'text': "What is the total cost of Octank Financial's property and equipment as of December 31, 2022?"},
     'traceId': '0a337e61-dd98-4149-a031-913c2ed9c27a-0'}}}},
 {'agentAliasId': 'TSTALIASID',
  'agentId': 'XNSPVCAHBV',
  'agentVersion': 'DRAFT',
  'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:533267284022:agent-alias/XNSPVCAHBV/TSTALIASID'}],
  'sessionId': '562ea3f7-0945-456a-8760-4b534e460061',
  'trace': {'orchestrationTrace': {'observation': {'knowledgeBaseLookupOutput': {'retrievedReferences': [{'content': {'text': "The portfolio is classified into

In [12]:
result['enhanced_trace']

[]